In [ ]:
# === 1. Imports ===
import sys
import os
sys.path.append(os.path.abspath("../"))  # Ensure src is discoverable

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

from src.load_data import load_train_data, save_processed_data
from src.preprocess.preprocessing_pipeline import PreprocessingPipeline


In [ ]:
# === 2. Load data ===
df = load_train_data()

In [ ]:
# === 3. Prepare features and target (before preprocessing) ===
# We need to split first, then preprocess separately to avoid leakage
X_raw = df.drop(columns=["price"])
y = df["price"]

In [ ]:
# === 4. Train-test split (on raw data) ===
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X_raw, y, test_size=0.2, random_state=42
)

In [ ]:
# === 4a. Preprocess training data ===
train_df = X_train_raw.copy()
train_df["price"] = y_train

pipeline = PreprocessingPipeline(
    use_log_target=False, 
    drop_low_importance=True,
    encode_data=True
)
train_processed = pipeline.fit_transform(train_df)
X_train = train_processed.drop(columns=["price"])
y_train = train_processed["price"]

In [ ]:
# === 4b. Preprocess test data (using fitted pipeline) ===
test_df = X_test_raw.copy()
test_df["price"] = y_test  # For consistency, though we won't use it
test_processed = pipeline.transform(test_df)
X_test = test_processed.drop(columns=["price"], errors='ignore')
y_test = test_processed["price"] if "price" in test_processed.columns else y_test

In [ ]:
# === 4c. Generate Out-of-Fold Predictions for Stacking ===
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(len(df))  # Store OOF predictions

for train_idx, val_idx in kf.split(df):
    # Get raw data for this fold
    train_fold_raw = df.iloc[train_idx].copy()
    val_fold_raw = df.iloc[val_idx].copy()
    
    # Create and fit pipeline on fold training data
    fold_pipeline = PreprocessingPipeline(
        use_log_target=False,
        drop_low_importance=True,
        encode_data=True
    )
    train_fold_processed = fold_pipeline.fit_transform(train_fold_raw)
    X_tr = train_fold_processed.drop(columns=["price"])
    y_tr = train_fold_processed["price"]
    
    # Transform validation fold
    val_fold_processed = fold_pipeline.transform(val_fold_raw)
    X_val = val_fold_processed.drop(columns=["price"], errors='ignore')
    y_val = val_fold_raw["price"].values  # Use original target
    
    # Train model
    fold_model = RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        max_features='sqrt',
        min_samples_leaf=1,
        random_state=42,
        n_jobs=-1
    )
    fold_model.fit(X_tr, y_tr)
    oof_preds[val_idx] = fold_model.predict(X_val)

# Save OOF predictions
pd.DataFrame({"rf_oof_pred": oof_preds}).to_csv("../results/rf_oof_train_preds.csv", index=False)
print("Saved RF OOF predictions to ../results/rf_oof_train_preds.csv")

In [ ]:
# === 5. Hyperparameter tuning using GridSearchCV ===
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    verbose=1,
    scoring='neg_root_mean_squared_error'
)

grid_search.fit(X_train, y_train)
rf_model = grid_search.best_estimator_

print("Best parameters:", grid_search.best_params_)

In [ ]:
# === 6. Predict and evaluate ===
y_pred = rf_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Random Forest RMSE: ${rmse:.2f}")
print(f"Random Forest R² Score: {r2:.3f}")

In [ ]:
# === 6b. Feature Importance Analysis ===
import matplotlib.pyplot as plt

# Create a DataFrame of feature importances
importances_df = pd.DataFrame({
    "Feature": X_train.columns,
    "Importance": rf_model.feature_importances_
}).sort_values(by="Importance", ascending=False)

# Display top features
print("\nTop 15 Features by Importance:")
print(importances_df.head(15))

# Plot top features
plt.figure(figsize=(10, 6))
plt.barh(importances_df.head(15)["Feature"][::-1], importances_df.head(15)["Importance"][::-1])
plt.xlabel("Importance")
plt.title("Top 15 Feature Importances")
plt.tight_layout()
plt.show()

In [ ]:
# === 8. Save the tuned model ===
import joblib
joblib.dump(rf_model, "../models/random_forest_tuned.joblib")


In [ ]:
# === 9. Load test data and preprocess ===
from src.load_data import load_test_data
test_df = load_test_data()
# Use the same pipeline that was fitted on training data
test_processed = pipeline.transform(test_df)


In [ ]:
# === 10. Predict and save Kaggle submission ===
test_preds = rf_model.predict(test_processed)
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "Actual": test_preds
})
submission.to_csv("../results/tuned_rf_submission.csv", index=False)
print("✅ Submission saved to: ../results/tuned_rf_submission.csv")
